# GT Educação

O GT Educação enviou uma série de formulários com indicadores desejados para o painel. A maioria desses indicadores estão disponíveis no Portal de Dados Abertos da Prefeitura de São Paulo (http://dados.prefeitura.sp.gov.br/).

Portanto, vamos carregar o módulo de downloads do portal de dados abertos.

In [ ]:
import os
import pandas as pd
from xlrd import xldate
from unidecode import unidecode
from dotenv import load_dotenv

from core.downloads import dadosabertos as da
from core.downloads import geosampa as gs

In [ ]:
load_dotenv()

# Extração e transformação inicial

## Número de alunos da Rede Municipal de Ensino

O primeiro indicador é o número de alunos da rede municipal, segmentado por nível educacional (Creche, Pré Escola, Ensino Fundamental I, Ensino Fundamental II, EJA I, EJA II, Ensino Médio, Ed. Prof.).

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/demanda-e-matriculas. Sabendo que o último trecho da url representa a id do conjunto de dados, vamos utilizá-la para fazer a extração dos dados do portal.

Primeiro, precisamos visualizar todos os recursos relativos a esse conjunto de dados. Nesse momento, podemos pegar todos os conjuntos em dezembro de 2024.

In [ ]:
pkg_name = 'demanda-e-matriculas'
resources = da.package_resources(pkg_name, '24')
resources

Sabendo o id do arquivo de dezembro de 2023, vamos carregar o recurso como um dataframe.

In [ ]:
resource_id = '23081b4e-7866-4c3e-843b-489354c00a3e'
mat_24 = da.load_resource(resource_id, pandas_kwargs=dict(header=[0,1]))
mat_24

O dataframe contem dados de matrículas, matrículas em processo e demanda não atendida. Como desejamos apenas matrículas, vamos remover as outras colunas.

In [ ]:
mat_24 = mat_24[['Distrito', 'Matrículas']]
mat_24

Agora, sabendo que temos apenas dados sobre matrículas, vamos ajustar as colunas.

In [ ]:
mat_24 = (mat_24
 .droplevel(0, axis='columns')
 .rename(columns={'Unnamed: 0_level_1': 'Distrito'}))

mat_24

As últimas linhas do dataframe são apenas dados de totalização e notas, então vamos excluí-las também.

In [ ]:
mat_24 = mat_24.loc[mat_24['Distrito'].str.lower()!='total']
mat_24 = mat_24.loc[~mat_24.iloc[:,1].isna()]
mat_24

Finalmente, vamos "despivotar" a tabela, transformando o nome das colunas em uma nova coluna chamada "Nível educacional" e unificando as colunas de valor sob uma coluna chamada matrículas.

In [ ]:
mat_24 = mat_24.melt(
    id_vars='Distrito',
    var_name='Nível Educacional',
    value_name='Matrículas'
)
mat_24

## Número de professores da Rede Municipal de Ensino

O segundo indicador solicitado é o número de professores da Rede Municipal de Ensino, com recortes por Raça, Sexo, Nível de Ensino (Educação Infantil, Ensino Fundamental etc.), Tipo de Unidade de atuação (EMEF, EMEI etc.).

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/microdados-servidores-perfil, porém os arquivos possuem informação sobre todos os servidores da SME, o que inclui os servidores administrativos, que precisarão ser excluídos.

Primeiro, precisamos visualizar todos os recursos relativos a esse conjunto de dados. Nesse momento, podemos pegar todos os conjuntos em dezembro de 2024.

In [ ]:
pkg_name = 'microdados-servidores-perfil'
resources = da.package_resources(pkg_name, '24')
resources

Aparentemente, os dados de perfil dos servidores possuem periodicidade anual, já que temos apenas um arquivo disponível para 2024.

In [ ]:
resource_id = resources[0]['id']
perf_24 = da.load_resource(resource_id)
perf_24

Com os dados baixados, vamos selecionar as colunas para atender às segmentações propostas no formulário.

In [ ]:
# Raça, Sexo, Nível de Ensino (Educação Infantil, Ensino Fundamental etc.), Tipo de Unidade de atuação (EMEF, EMEI etc.).

perf_cols = [
    'CD_SEXO',
    'CD_RACA_COR',
    'DC_RACA_COR',
    'CD_DEF',
    'DC_SIT_FUNC',
    'CD_CARGO_BASE',
    'DC_CARGO_BASE',
    'DT_INICIO_CARGO_BASE',
    'CD_CARGO_ATUAL',
    'DC_CARGO_ATUAL',
    'CD_AREA_ATUACAO_BASE',
    'DC_AREA_ATUACAO_BASE',
    'CD_UNIDADE_ATUAL',
    'TP_UNIDADE_ATUAL',
    'DC_UNIDADE_ATUAL',
]

perf_24 = perf_24.loc[:, perf_cols]
perf_24

O formulário solicita também a exclusão de professores aposentados e servidores administrativos do conjunto de dados. Apesar de existir uma coluna com um valor "aposentado", a `ACUMULO_SITUACAO_SEGUNDO_VINCULO`, essa coluna parece se referir à situação do segundo vínculo, e não do vínculo com a PMSP. Além disso, a lista de servidores aposentados da Secretaria Municipal de Educação tem mais de 94 mil registros, então esses registros provavelmente não estão nos dados de perfil dos servidores.

Para excluir os servidores administrativos, vamos primeiro avaliar quais cargos estão presentes na base.

In [ ]:
df_cargos = perf_24[['CD_CARGO_BASE', 'DC_CARGO_BASE']].copy()
df_cargos = df_cargos.drop_duplicates().reset_index(drop=True)

df_cargos

Dado a quantidade de cargos, vamos fitrar a lista com base em uma regra geral, mantendo apenas os cargos que possual a string "PROF" em sua descrição.

In [ ]:
filtro_prof = df_cargos['DC_CARGO_BASE'].str.lower().str.contains('prof.', regex=False)
df_cargos_prof = df_cargos.loc[filtro_prof]

df_cargos_prof

Por último, filtramos a base geral com base nessa lista de cargos.

In [ ]:
perf_24 = (
    perf_24
    .loc[
        perf_24['DC_CARGO_BASE'].isin(df_cargos_prof['DC_CARGO_BASE'].tolist())
    ]
)

perf_24

## Número de Unidades Escolares

O terceiro indicador solicitado é o número de Unidades Escolares da Rede Municipal de Ensino, com recortes por Tipo de Escola (EMEI, EMEF etc.) e Distrito.

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/cadastro-de-escolas-municipais-conveniadas-e-privadas.

Primeiro, precisamos visualizar todos os recursos relativos a esse conjunto de dados. Nesse momento, podemos pegar todos os conjuntos em dezembro de 2024.

In [ ]:
pkg_name = 'cadastro-de-escolas-municipais-conveniadas-e-privadas'
resources = da.package_resources(pkg_name, '24')
resources

In [ ]:
resource_id = resources[0]['id']
escolas_24 = da.load_resource(resource_id)
escolas_24

Os dados das escolas também serão usados com outros conjuntos de dados para regionalizá-los, então vamos manter também algumas colunas de identificação das escolas para facilitar esse uso o posterior. Vamos também filtrar apenas as escolas ativas.

In [ ]:
escolas_cols = ['CODESC',
                'TIPOESC',
                'NOMES',
                'SUBPREF',
                'DISTRITO',
                'CODINEP',
                'CD_CIE',
                'NOME_ANT',
                'REDE']

escolas_24 = escolas_24.loc[:, escolas_cols]

escolas_24

As unidades escolares também recebeu instruções de filtro para que os número fossem adequados ao informado no relatório de função. Devemos excluir as escolas de tipo `CEU AT COM` e `ESC.PART.`.

In [ ]:
escolas_24['TIPOESC'].value_counts()

In [ ]:
escolas_24 = escolas_24[~escolas_24['TIPOESC'].isin(['CEU AT COM', 'ESC.PART.'])]

escolas_24

## Perfil dos alunos

O quinto indicador solicitado é o número de alunos da Rede Municipal de Ensino, com recortes por Raça, Sexo, existência de Necessidades Especiais, Distrito, Rede de Ensino (Administração Direta ou Rede Conveniada/Parceira).

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/perfil-dos-educandos-cor-raca-idade-sexo-necessidades-educacionais-especiais.

Primeiro, precisamos visualizar todos os recursos relativos a esse conjunto de dados. Nesse momento, podemos pegar todos os conjuntos em dezembro de 2024.

In [ ]:
pkg_name = 'perfil-dos-educandos-cor-raca-idade-sexo-necessidades-educacionais-especiais'
resources = da.package_resources(pkg_name, '24')
resources

Aparentemente, os dados de perfil dos educandos possuem periodicidade anual, já que temos apenas um arquivo disponível para 2023.

In [ ]:
resource_id = resources[0]['id']
alunos_24 = da.load_resource(resource_id)
alunos_24

In [ ]:
total_alunos_24_pre_filtros = alunos_24['Qtde'].sum()
total_alunos_24_pre_filtros

### Ajustando alunos de `EDPROF`

Para manter a quantidade de matrículas compatível com o relatório de função, vamos manter apenas algumas modalidades de ensino, de modo que não sejam consideradas matrículas de atividades em contraturno e similares. Além disso, a soma de alunos de `EDPROF` é diferente do último valor informado pela SME em auditoria. O valor informado pela SME foi de 1.879 matrículas, porém o total com base nos dados de `Perfil dos alunos` é 4.722. Vamos tentar entender de onde vem essa distorção.

In [ ]:
(
    alunos_24
    .loc[alunos_24['MODAL'] == 'EDPROF', ['TIPOESC', 'Qtde']]
    .groupby('TIPOESC')
    .sum().reset_index()
)

In [ ]:
1445+46+388

Vemos que a soma dos alunos de `E TEC` + `EMEBS` + `EMEFM` coincide exatamente com o número de matrículas informado pela SME. Após apresentar a situação para o Coordenador do GT Educação, decidiu-se por excluir os registros de `EDPROF` de `CEU AT COM` e `CMCT`.

In [ ]:
filtro_alunos_24 = (
    (alunos_24['MODAL'] != 'EDPROF') |
    (~alunos_24['TIPOESC'].isin(['CEU AT COM', 'CMCT']))
)

In [ ]:
alunos_24 = alunos_24.loc[filtro_alunos_24]
alunos_24

Finalmente, vamos conferir se o filtro funcionou corretamente.

In [ ]:
(
    alunos_24
    .loc[alunos_24['MODAL'] == 'EDPROF', ['TIPOESC', 'Qtde']]
    .groupby('TIPOESC')
    .sum().reset_index()
)

In [ ]:
total_alunos_24_pos_filtros = alunos_24['Qtde'].sum()
total_alunos_24_pos_filtros

In [ ]:
total_alunos_24_pre_filtros - total_alunos_24_pos_filtros

Vemos que todos os alunos de `EDPROF` que deviam estar presentes estão e a diferença antes e depois dos filtros é exatamente a quantidade de alunos de `CEU AT COM` e `CMCT`, então o filtro funcionou corretamente.

### Ajustando alunos de `ESPEC`

A soma de alunos de `ESPEC` também é diferente do último valor informado pela SME em auditoria. O valor informado pela SME foi de 693 matrículas, porém o total com base nos dados de `Perfil dos alunos` é 766. Vamos tentar entender de onde vem essa distorção.

In [ ]:
(
    alunos_24
    .loc[alunos_24['MODAL'] == 'ESPEC', ['TIPOESC', 'Qtde']]
    .groupby('TIPOESC')
    .sum().reset_index()
)

Vemos que a soma dos alunos de `EMEBS` coincide exatamente com o número de matrículas informado pela SME. Após apresentar a situação para o Coordenador do GT Educação, decidiu-se por excluir os registros de `CEU EMEF` e `CEU EMEI`.

In [ ]:
total_alunos_24_pre_filtros = alunos_24['Qtde'].sum()
total_alunos_24_pre_filtros

In [ ]:
filtro_alunos_24 = (
    (alunos_24['MODAL'] != 'ESPEC') |
    (~alunos_24['TIPOESC'].isin(['CEU EMEF', 'CEU EMEI']))
)

In [ ]:
alunos_24 = alunos_24.loc[filtro_alunos_24]
alunos_24

Finalmente, vamos conferir se o filtro funcionou corretamente.

In [ ]:
(
    alunos_24
    .loc[alunos_24['MODAL'] == 'ESPEC', ['TIPOESC', 'Qtde']]
    .groupby('TIPOESC')
    .sum().reset_index()
)

In [ ]:
total_alunos_24_pos_filtros = alunos_24['Qtde'].sum()
total_alunos_24_pos_filtros

In [ ]:
total_alunos_24_pre_filtros - total_alunos_24_pos_filtros

Vemos que todos os alunos de `ESPEC` que deviam estar presentes estão e a diferença antes e depois dos filtros é exatamente a quantidade de alunos de `CEU EMEF` e `CEU EMEI`, então o filtro funcionou corretamente.

In [ ]:
alunos_24['MODAL'].unique()

In [ ]:
modalidades = ['CRECHE', 'ESPEC', 'PRE',
               'CONVEE','EDPROF', 'EJA', 'FUND', 'MEDIO', 'MOVA']

alunos_24 = alunos_24[alunos_24['MODAL'].isin(modalidades)]
alunos_24

In [ ]:
alunos_cols = ['DISTRITO',
               'REDE',
               'MODAL',
               'SEXO',
               'NEE',
               'RACA',
               'Qtde']

alunos_24 = alunos_24[alunos_cols]

alunos_24

In [ ]:
alunos_24['Qtde'].sum()

O total de alunos está exatamente igual ao total esperado, que é de 1.027.308.

## Número de alunos estrangeiros

O sétimo indicador é o número de alunos estrangeiros da rede municipal, segmentado por País de Procedência, Distrito, Tipo de Rede (Direta ou Conveniada/Parceira), Etapa (Creche, fundamental, EJA etc.).

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/educandos-estrangeiros-por-nacionalidade. Sabendo que o último trecho da url representa a id do conjunto de dados, vamos utilizá-la para fazer a extração dos dados do portal. Nesse momento, podemos pegar todos os conjuntos em dezembro de 2024.

Primeiro, precisamos visualizar todos os recursos relativos a esse conjunto de dados.

In [ ]:
pkg_name = 'educandos-estrangeiros-por-nacionalidade'
resources = da.package_resources(pkg_name, '24')
resources

In [ ]:
resource_id = resources[0]['id']
alunos_est_2024 = da.load_resource(resource_id)
alunos_est_2024

In [ ]:
# Recortes: País de Procedência, Distrito, Tipo de Rede (Direta ou Conveniada/Parceira), Etapa (Creche, fundamental, EJA etc.)
alunos_est_cols = ['DISTRITO',
                   'COD_PAIS',
                   'NOME_PAIS',
                   'ETAPA',
                   'REDE',
                   'QTDE']

alunos_est_2024 = alunos_est_2024[alunos_est_cols]

alunos_est_2024

## Número de alunos beneficiários do Bolsa Família

O oitavo indicador é o número de alunos beneficiários do Bolsa Família da rede municipal, segmentado por Distrito e nível de ensino.

O conjunto de dados que contém esse indicador está disponível em http://dados.prefeitura.sp.gov.br/dataset/beneficiarios-programa-bolsa-familia-creches-municipais-e-conveniadas. Sabendo que o último trecho da url representa a id do conjunto de dados, vamos utilizá-la para fazer a extração dos dados do portal.

As notas (que podem ser vistas na saída abaixo) desse conjunto de dados incluem a informação de que a estrutura do conjunto mudou a partir de 2024 e, por isso, foi criado um novo conjunto de dados com a estrutura mais recente, disponível em http://dados.prefeitura.sp.gov.br/dataset/beneficiarios-do-programa-bolsa-familia-por-etapa-de-ensino-da-rede-municipal-de-educacao. Vamos utilizar essa versão mais recente e os dados de julho de 2024, assim como o número de alunos estrangeiros.

In [ ]:
pkg_name = 'beneficiarios-programa-bolsa-familia-creches-municipais-e-conveniadas'
pkg = da.package_show(pkg_name)
print(pkg['notes'])

In [ ]:
pkg_name = 'beneficiarios-do-programa-bolsa-familia-por-etapa-de-ensino-da-rede-municipal-de-educacao'
resources = da.package_resources(pkg_name, '24')
resources

A última data disponível permanece sendo a de julho de 2024, então vamos utilizar essa data para extrair os dados.

In [ ]:
resource_id = resources[2]['id']
alunos_bf_2024 = da.load_resource(resource_id)
alunos_bf_2024

In [ ]:
# Recortes: Alunos Beneficiários do Bolsa-Família por distrito, por nível de Ensino
alunos_bf_cols = ['CODEOL',
                   'TIPO',
                   'UNIDADE',
                   'DIRETORIA',
                   'DISTRITO',
                   'CRECHE',
                   'PRE ESCOLA',
                   'ENSINO FUNDAMENTAL',
                   'EDUCAÇÃO PROFISSIONAL',
                   'ENSINO MEDIO',
                   'EDUCAÇÃO ESPECIAL']

alunos_bf_2024 = alunos_bf_2024[alunos_bf_cols]

alunos_bf_2024

In [ ]:
alunos_bf_id_vars = alunos_bf_cols[0:5]

alunos_bf_2024 = alunos_bf_2024.melt(id_vars=alunos_bf_id_vars,
                    var_name='NIVEL',
                    value_name='QTDE')

alunos_bf_2024

## Distritos e Subprefeituras

Para agregarmos os dados em Subprefeituras, precisamos de uma lista de distritos e subprefeituras. A lista de distritos e subprefeituras pode ser obtida no GeoSampa, que é o portal de dados geográficos da Prefeitura de São Paulo.

In [ ]:
gs.get_capabilities('distrito')

In [ ]:
df_dist = gs.get_features('geoportal:distrito_municipal')
df_dist

In [ ]:
gs.get_capabilities('subprefeitura')

In [ ]:
df_subs = gs.get_features('geoportal:subprefeitura')
df_subs

## CSV de Subprefeituras do Qlik

In [ ]:
url_subs = os.environ.get('CSV_SUBPREFEITURAS_QLIK')
df_subs_qlik = pd.read_csv(url_subs)
df_subs_qlik

In [ ]:
df_subs_qlik = df_subs_qlik[['sub.CODIGO', 'sub.NOME']]
df_subs_qlik

## Orçamento previsto/liquidado na função Educação

Vamos começar coletando os dados orçamentários do site de execução orçamentária da Secretaria da Fazenda.

In [ ]:
url_orcamento = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2024/basedadosexecucao_1224.csv'
df_orcamento = pd.read_csv(url_orcamento,
                           sep=';',
                           decimal=',',
                           encoding='latin1',
                           dtype=str)
df_orcamento

In [ ]:
for col in [col for col in df_orcamento.columns if 'Vl' in col]:
    df_orcamento[col] = df_orcamento[col].str.replace(',', '.').astype(float)
df_orcamento['DataExtracao'] = pd.to_datetime(df_orcamento['DataExtracao'], format='%d/%m/%Y')
df_orcamento

In [ ]:
df_orcamento = df_orcamento.loc[df_orcamento['Cd_Funcao']=='12']
df_orcamento

## Orçamento regionalizado na função Educação

O orçamento é regionalizado apenas na liquidação, então não é possível obter o orçamento previsto por subprefeitura, mas é possível obter o orçamento liquidado.

In [ ]:
url_orcamento_r = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2024/basedadosDA_1224.csv'
df_orcamento_r = pd.read_csv(url_orcamento_r,
                           sep=';',
                           decimal=',',
                           encoding='latin1',
                           dtype=str)
df_orcamento_r

In [ ]:
df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'] = df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].str.replace(',', '.').astype(float)
# df_orcamento_r['DATA_EXTRAÇÃO'] = pd.to_datetime(df_orcamento_r['DATA_EXTRAÇÃO'], format='%d/%m/%Y')
df_orcamento_r

In [ ]:
df_orcamento_r = df_orcamento_r.loc[df_orcamento_r['CÓDIGO_FUNÇÃO']=='12']
df_orcamento_r = df_orcamento_r.loc[df_orcamento_r['ANO_LIQUIDAÇÃO']=='2024']
df_orcamento_r

Vamos conferir qual o percentual do orçamento está presente na tabela de detalhamento e regionalizado a nível de subprefeitura.

In [ ]:
df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].sum()/df_orcamento['Vl_Liquidado'].sum()

53,9% de detalhamento não é um percentual tão bom, mas vamos checar a regionalização.

In [ ]:
(
    df_orcamento_r
    .assign(regionalizado=df_orcamento_r['SUBPREFEITURA'].str.contains('Supra')==False)
    .groupby('regionalizado')
    ['VALOR_DETALHAMENTO_AÇÃO'].sum()/df_orcamento['Vl_Liquidado'].sum()
)

O percentual de regionalização é de 27,5%, o que é bem baixo. Apesar de não parecer tão satisfatório, não perdemos nada ao mantê-los, então vamos manter as informações de regionalização também.

# Transformação e mesclagem de dados

Para facilitar o trabalho dos dados no Qlik Sense, vamos avaliar como é a melhor forma de retrabalhar os dados de acordo com o paradigma fato-dimensão.

## Nível Escolar

A primeira dimensão que parece fazer sentido é o Nível escolar. Vamos avaliar como aparece em cada tabela.

In [ ]:
mat_24['Nível Educacional'].value_counts()

In [ ]:
perf_24[['CD_AREA_ATUACAO_BASE', 'DC_AREA_ATUACAO_BASE']].value_counts()

In [ ]:
alunos_24['MODAL'].value_counts()

In [ ]:
alunos_est_2024['ETAPA'].value_counts()

In [ ]:
alunos_bf_2024['NIVEL'].value_counts()

Existe uma tabela de correspondência relacionada a essas categorias fornecida no último relatório de Gestão da Função Educação. A tabela é a seguinte:

| Nível Educacional | Tipo de escola |
| --- | --- |
| Educação Infantil | Creches |
| Educação Infantil | Pré-escola |
| Ensino Fundamental | EMEF |
| Ensino Médio | EMEFM |
| Educação de Jovens e Adultos | EJA/CIEJA |
| Educação de Jovens e Adultos | MOVA (Parceria) |
| Educação de Jovens e Adultos | Educação Profissional |
| Educação Especial | EMEBS |
| Educação Especial | Educação Especial (Parceira) |


Essa tabela permite a padronização de quase todas as tabelas, exceto a de perfil dos professores. Para as outras, vamos utilizar a tabela como base para a criação de um dicionário que será utilizado como *mapper* em cada uma das tabelas.

In [ ]:
mat_23_nivel = {
    'Creche': 'Educação Infantil',
    'Pré Escola': 'Educação Infantil',
    'Ens. Fund.I': 'Ensino Fundamental',
    'Ens. Fund.II': 'Ensino Fundamental',
    'EJA I': 'Educação de Jovens e Adultos',
    'EJA II': 'Educação de Jovens e Adultos',
    'Ens. Médio': 'Ensino Médio',
    'Ed. Prof.': 'Educação de Jovens e Adultos'
}

mat_24['Nível padronizado'] = mat_24['Nível Educacional'].map(mat_23_nivel)
mat_24['Nível padronizado'].value_counts()

In [ ]:
alunos_23_nivel = {
    'CRECHE': 'Educação Infantil',
    'PRE': 'Educação Infantil',
    'FUND': 'Ensino Fundamental',
    'EJA': 'Educação de Jovens e Adultos',
    'MOVA': 'Educação de Jovens e Adultos',
    'MEDIO': 'Ensino Médio',
    'EDPROF': 'Educação de Jovens e Adultos',
    'ESPEC': 'Educação Especial',
    'CONVEE': 'Educação Especial',
    'ATCOMP': '?',
    'REC': '?',
    'SAAI': '?'
}

alunos_24['Nível padronizado'] = alunos_24['MODAL'].map(alunos_23_nivel)
alunos_24['Nível padronizado'].value_counts()

In [ ]:
alunos_est_2024_nivel = {
    'CRECHE': 'Educação Infantil',
    'PRE': 'Educação Infantil',
    'FUND': 'Ensino Fundamental',
    'EJA': 'Educação de Jovens e Adultos',
    'MOVA': 'Educação de Jovens e Adultos',
    'MEDIO': 'Ensino Médio',
    'EDPROF': 'Educação de Jovens e Adultos',
    'ESPEC': 'Educação Especial',
    'CONVEE': 'Educação Especial'
}

alunos_est_2024['Nível padronizado'] = alunos_est_2024['ETAPA'].map(alunos_est_2024_nivel)
alunos_est_2024['Nível padronizado'].value_counts()

In [ ]:
alunos_bf_2024_nivel = {
    'CRECHE': 'Educação Infantil',
    'PRE ESCOLA': 'Educação Infantil',
    'ENSINO FUNDAMENTAL': 'Ensino Fundamental',
    'ENSINO MEDIO': 'Ensino Médio',
    'EDUCAÇÃO PROFISSIONAL': 'Educação de Jovens e Adultos',
    'EDUCAÇÃO ESPECIAL': 'Educação Especial'
}

alunos_bf_2024['Nível padronizado'] = alunos_bf_2024['NIVEL'].map(alunos_bf_2024_nivel)
alunos_bf_2024['Nível padronizado'].value_counts()

## Padronização dos nomes de Distritos

Os dados de matrículas, escolas, educandos, educandos estrangeiros e educandos beneficiários do Bolsa Família possuem uma coluna chamada "Distrito" que contém o nome do distrito onde a escola está localizada.

Já os dados de servidores não possuem dados de distrito, mas possuem o nome da escola, que pode ser utilizada para buscar o distrito correspondente. Antes de unir as tabelas para fazer isso, vamos padronizar os nomes dos distritos de acordo com os dados do geosampa.

### Número de Unidades Escolares

In [ ]:
(
    escolas_24
    .loc[~escolas_24['DISTRITO'].isin(df_dist['nm_distrito_municipal'])]
    ['DISTRITO']
    .unique().tolist()
)

### Perfil dos alunos

In [ ]:
(
    alunos_24
    .loc[~alunos_24['DISTRITO'].isin(df_dist['nm_distrito_municipal'])]
    ['DISTRITO']
    .unique().tolist()
)

### Número de alunos estrangeiros

In [ ]:
(
    alunos_est_2024
    .loc[~alunos_est_2024['DISTRITO'].isin(df_dist['nm_distrito_municipal'])]
    ['DISTRITO']
    .unique().tolist()
)

### Número de alunos beneficiários do Bolsa Família

In [ ]:
(
    alunos_bf_2024
    .loc[~alunos_bf_2024['DISTRITO'].isin(df_dist['nm_distrito_municipal'])]
    ['DISTRITO']
    .unique().tolist()
)

Nenhum dos conjuntos de dados possui distritos que não estejam na lista do GeoSampa.

Agora, vamos unir os dados de servidores com os dados de escolas para adicionar a coluna de distrito aos dados de servidores.

### Número de professores da Rede Municipal de Ensino

Primeiro, vamos criar uma coluna com o tipo de escola e o nome da escola nos dois dataframes, para que possamos unir os dados de servidores com os dados de escolas.

In [ ]:
escolas_24['NOMES_COMPLETO'] = (
    escolas_24['TIPOESC'].str.strip().str.upper()
    + ' ' + escolas_24['NOMES'].str.strip().str.upper()
)
escolas_24['NOMES_COMPLETO']

In [ ]:
perf_24['DC_UNIDADE_ATUAL_COMPLETA'] = (
    perf_24['TP_UNIDADE_ATUAL'].str.strip().str.upper()
    + ' ' + perf_24['DC_UNIDADE_ATUAL'].str.strip().str.upper()
)
perf_24['DC_UNIDADE_ATUAL_COMPLETA']

In [ ]:
perf_24 = (
    perf_24
    .merge(escolas_24[['NOMES_COMPLETO', 'DISTRITO']],
           left_on=['DC_UNIDADE_ATUAL_COMPLETA'],
           right_on=['NOMES_COMPLETO'],
           how='left')
)

perf_24

Vamos avaliar se todas as unidades escolares foram encontradas no dataframe de unidades.

In [ ]:
(
    perf_24
    .query('DISTRITO.isnull()')
)

In [ ]:
(
    perf_24
    .query('DISTRITO.isna()')
    .shape[0]
    /perf_24.shape[0]
)

Cerca de 3,6% dos professores não tiveram as unidades escolares encontradas. Vamos avaliar quais são essas unidades.

In [ ]:
(
    perf_24
    .query('DISTRITO.isna()')
    [['TP_UNIDADE_ATUAL', 'DC_UNIDADE_ATUAL']]
    .sort_values(['TP_UNIDADE_ATUAL', 'DC_UNIDADE_ATUAL'])
    .drop_duplicates()
)

Por ora, vamos manter dessa forma e, caso seja necessário, podemos avaliar como tratar esses casos numa atualização futura.

## Adicionando as Subprefeituras

Agora, vamos adicionar a coluna de subprefeitura a todos os dataframes.

### Associando distritos e subprefeituras

In [ ]:
df_dist.sort_values('cd_identificador_subprefeitura').head(2)

In [ ]:
df_subs.sort_values('cd_identificador_subprefeitura').head(2)

In [ ]:
df_subs.sort_values('cd_subprefeitura').head(2)

Os dados de distritos e subprefeituras não estão completamente compatíveis, então precisamos de um tratamento adicional.

O dataframe de distritos possui uma coluna chamada `cd_identificador_subprefeitura`, mas não corresponde exatamente à coluna `cd_identificador_subprefeitura` do dataframe de subprefeituras. Ele corresponde parcialmente à coluna `cd_subprefeitura` do dataframe de subprefeituras, mas não é exatamente a mesma, porque esta última possui um zero à esquerda que não está presente no dataframe de distritos. Portanto, vamos criar uma coluna com o código de subprefeitura no dataframe de distritos, que será o código de subprefeitura com o zero à esquerda.

In [ ]:
df_dist['cd_subprefeitura'] = (
    df_dist['cd_identificador_subprefeitura'].astype(str)
    .str.zfill(2)
)

df_dist.sort_values('cd_subprefeitura').head(2)

Agora, vamos associar os distritos e subprefeituras.

In [ ]:
df_dist_sub = df_dist.merge(
    df_subs[['cd_subprefeitura', 'nm_subprefeitura']],
    on='cd_subprefeitura',
    how='left'
)
df_dist_sub.sort_values('cd_subprefeitura')

Aparentemente, os dados de distritos e subprefeituras estão compatíveis, então vamos adicionar a coluna de subprefeitura aos outros dataframes.

### Número de Unidades Escolares

In [ ]:
escolas_24 = escolas_24.merge(
    df_dist_sub[['nm_distrito_municipal', 'cd_subprefeitura', 'nm_subprefeitura']],
    left_on='DISTRITO',
    right_on='nm_distrito_municipal',
    how='left'
)

escolas_24

In [ ]:
escolas_24[escolas_24['nm_subprefeitura'].isna()]

### Perfil dos alunos

In [ ]:
alunos_24 = alunos_24.merge(
    df_dist_sub[['nm_distrito_municipal', 'cd_subprefeitura', 'nm_subprefeitura']],
    left_on='DISTRITO',
    right_on='nm_distrito_municipal',
    how='left'
)

alunos_24

In [ ]:
alunos_24[alunos_24['nm_subprefeitura'].isna()]

### Número de alunos estrangeiros

In [ ]:
alunos_est_2024 = alunos_est_2024.merge(
    df_dist_sub[['nm_distrito_municipal', 'cd_subprefeitura', 'nm_subprefeitura']],
    left_on='DISTRITO',
    right_on='nm_distrito_municipal',
    how='left'
)

alunos_est_2024

In [ ]:
alunos_est_2024[alunos_est_2024['nm_subprefeitura'].isna()]

### Número de alunos beneficiários do Bolsa Família

In [ ]:
alunos_bf_2024 = alunos_bf_2024.merge(
    df_dist_sub[['nm_distrito_municipal', 'cd_subprefeitura', 'nm_subprefeitura']],
    left_on='DISTRITO',
    right_on='nm_distrito_municipal',
    how='left'
)

alunos_bf_2024

In [ ]:
alunos_bf_2024[alunos_bf_2024['nm_subprefeitura'].isna()]

### Número de professores da Rede Municipal de Ensino

In [ ]:
perf_24 = perf_24.merge(
    df_dist_sub[['nm_distrito_municipal', 'cd_subprefeitura', 'nm_subprefeitura']],
    left_on='DISTRITO',
    right_on='nm_distrito_municipal',
    how='left'
)

perf_24

In [ ]:
perf_24[perf_24['nm_subprefeitura'].isna() & ~perf_24['DISTRITO'].isna()]

## Adicionando tempo de atuação dos educadores na SME

In [ ]:
ref_date = pd.to_datetime('2023-12-31')
ref_date

No ano de 2024, a coluna de data de início de atuação dos educadores foi disponibilizada com o formato numérico usado pelo Excel, que é o número de dias desde 30/12/1899. Vamos converter essa coluna para o formato de data.

In [ ]:
perf_24['DT_INICIO_CARGO_BASE'] = perf_24['DT_INICIO_CARGO_BASE'].apply(
    lambda x: pd.to_datetime(xldate.xldate_as_datetime(x, 0))
    if isinstance(x, (int, float)) else pd.NaT
)

perf_24

In [ ]:
perf_24['TEMPO_ATUACAO_SME'] =  ref_date - perf_24['DT_INICIO_CARGO_BASE']

perf_24

In [ ]:
perf_24['DIAS_ATUACAO_SME'] =  perf_24['TEMPO_ATUACAO_SME'].dt.days

perf_24

In [ ]:
perf_24['ANOS_ATUACAO_SME'] =  perf_24['TEMPO_ATUACAO_SME'].dt.days / 365.25
perf_24['ANOS_ATUACAO_SME'] = round(perf_24['ANOS_ATUACAO_SME'], 2)

perf_24

In [ ]:
perf_24 = perf_24.drop(columns='TEMPO_ATUACAO_SME')
perf_24

## Adicionando chave composta nível padronizado e Subprefeitura

### Criando tabela de níveis padronizados e subprefeituras

In [ ]:
nivel_padronizado = pd.DataFrame(
    data=['Educação Infantil',
          'Ensino Fundamental',
          'Ensino Médio',
          'Educação de Jovens e Adultos',
          'Educação Especial'],
    columns=['Nível padronizado']
)

nivel_padronizado

In [ ]:
nivel_padronizado = (
    nivel_padronizado
    .merge(df_subs['nm_subprefeitura'], how='cross')
)

nivel_padronizado

In [ ]:
nivel_padronizado['nivel_subprefeitura'] = (
    nivel_padronizado['Nível padronizado']
    + '|'
    + nivel_padronizado['nm_subprefeitura']
)

nivel_padronizado

### Adicionando a coluna de nível padronizado e subprefeitura aos outros dataframes

In [ ]:
alunos_24 = alunos_24.merge(
    nivel_padronizado,
    left_on=['Nível padronizado', 'nm_subprefeitura'],
    right_on=['Nível padronizado', 'nm_subprefeitura'],
    how='left'
)

alunos_24

In [ ]:
alunos_est_2024 = alunos_est_2024.merge(
    nivel_padronizado,
    left_on=['Nível padronizado', 'nm_subprefeitura'],
    right_on=['Nível padronizado', 'nm_subprefeitura'],
    how='left'
)

alunos_est_2024

In [ ]:
alunos_bf_2024 = alunos_bf_2024.merge(
    nivel_padronizado,
    left_on=['Nível padronizado', 'nm_subprefeitura'],
    right_on=['Nível padronizado', 'nm_subprefeitura'],
    how='left'
)

alunos_bf_2024

## Padronizando outras colunas comuns

### Rede conveniada

Os dados de diversas tabelas trazem uma coluna de categorização da rede em Direta ou Conveniada. Porém, o termo mais apropriado para conveniada, segundo o GT Educação, é "Parceira". Por isso, vamos fazer essa substituição em todas as tabelas que apresentam essa coluna.

In [ ]:
escolas_24['REDE'].value_counts()

In [ ]:
alunos_24['REDE'].value_counts()

In [ ]:
alunos_est_2024['REDE'].value_counts()

Todas as tabelas usam a mesma sigla, portanto podemos utilizar o mesmo dicionário para redefinir os valores.

In [ ]:
rede_mapper = {
    'DIR': 'Direta',
    'CON': 'Parceira'
}
alunos_24['DESC_REDE'] = alunos_24['REDE'].map(rede_mapper)
escolas_24['DESC_REDE'] = escolas_24['REDE'].map(rede_mapper)
alunos_est_2024['DESC_REDE'] = alunos_est_2024['REDE'].map(rede_mapper)

### Sexo

In [ ]:
perf_24['CD_SEXO'].value_counts()

In [ ]:
alunos_24['SEXO'].value_counts()

In [ ]:
sexo_mapper = {
    'M': 'Masculino',
    'F': 'Feminino',
    'm': 'Masculino',
    'f': 'Feminino'
}

In [ ]:
perf_24['DESC_SEXO'] = perf_24['CD_SEXO'].map(sexo_mapper)
alunos_24['DESC_SEXO'] = alunos_24['SEXO'].map(sexo_mapper)

### Raça/Cor

In [ ]:
perf_24['DC_RACA_COR'].value_counts()

In [ ]:
alunos_24['RACA'].value_counts()

In [ ]:
raca_mapper = {
    'BRANCA': 'Branca',
    'PRETA': 'Preta',
    'PARDA': 'Parda',
    'AMARELA': 'Amarela',
    'INDIGENA': 'Indígena',
    'NÃO DECLARADA': 'Recusou informar',
    'NAO INFORMADA': 'Recusou informar',
    'RECUSOU INFORMAR': 'Recusou informar',
    'RECUSOU INFORMA': 'Recusou informar'
}

In [ ]:
perf_24['DESC_RACA'] = perf_24['DC_RACA_COR'].map(raca_mapper)
alunos_24['DESC_RACA'] = alunos_24['RACA'].map(raca_mapper)

In [ ]:
perf_24['DESC_RACA'].value_counts()

In [ ]:
alunos_24['DESC_RACA'].value_counts()

### País de alunos estrangeiros

Para os nomes de países, vamos nos valer de uma lista já inspecionada e validada para manter os nomes em um padrão que o Qlik Sense aceite como nome válido para uma localização.

In [ ]:
paises_mapper = {
    'ESTADOS FEDERADOS DA MICRONESIA': 'MICRONÉSIA',
    'BÓSNIA HERZEGOVINA': 'BÓSNIA E HERZEGOVINA',
    'ABISSÍNIA': 'ETIÓPIA',
    'BONAIRE SAINT EUSTATIUS E SABA': 'PAÍSES BAIXOS CARIBENHOS',
    'ESTADOS UNIDOS DA AMÉRICA (EUA)': 'ESTADOS UNIDOS',
    'BIRMÂNIA': 'MYANMAR',
    'ANTÁRTICO ARGENTINO': 'ANTÁRTIDA ARGENTINA',
    'CHECHEN INGUSTH': 'RÚSSIA',
    'REPÚBLICA CENTRO AFRICANA': 'REPÚBLICA CENTRO-AFRICANA'
}

In [ ]:
alunos_est_2024['NOME_PAIS'].value_counts().sum()

In [ ]:
alunos_est_2024['NOME_PAIS'] = alunos_est_2024['NOME_PAIS'].replace(paises_mapper)
alunos_est_2024['NOME_PAIS'].value_counts().sum()

## Orçamento da função Educação

Para o orçamento, além de padronizar os nomes de subprefeituras e tipos de dados das métricas, precisaremos também adaptar os dados para compatibilizar o orçamento regionalizado e não realizado. Para isso, vamos fazer o seguinte:

1. Classificar o orçamento detalhado por nível de regionalização nas seguintes categorias: subprefeitura, região e não regionalizável;
1. Agrupar o restante do orçamento não detalhado e manter apenas o orçamento inicial, atualizado e liquidado;
1. Subtrair o total do orçamento detalhado do orçamento não detalhado e classificar o nível de regionalização como não regionalizado;
1. Unir os dois dataframes de orçamento de acordo com as dimensões mantidas.

### Orçamento regionalizado

In [ ]:
df_orcamento_r.head(1)

In [ ]:
cols_orcamento_r = ['CÓDIGO_SUBFUNÇÃO', 'DESCRIÇÃO_SUBFUNÇÃO',
                    'CÓDIGO_FONTE', 'REGIÃO',
                    'SUBPREFEITURA', 'TIPO_REGIONALIZAÇÃO']

cols_orcamento_r_vl = ['VALOR_DETALHAMENTO_AÇÃO']

df_orcamento_r = df_orcamento_r[cols_orcamento_r + cols_orcamento_r_vl]
df_orcamento_r

In [ ]:
df_orcamento_r['TIPO_REGIONALIZAÇÃO'].value_counts()

In [ ]:
df_orcamento_r.loc[df_orcamento_r['TIPO_REGIONALIZAÇÃO'].isna(), 'TIPO_REGIONALIZAÇÃO'] = 'Despesa Não-Regionalizável'
df_orcamento_r

In [ ]:
df_orcamento_r['TIPO_REGIONALIZAÇÃO'].value_counts()

In [ ]:
df_orcamento_r = df_orcamento_r.groupby(cols_orcamento_r).sum().round(2).reset_index()

df_orcamento_r

In [ ]:
df_orcamento_r.loc[
    ~df_orcamento_r['REGIÃO'].str.contains('Supra', na=False),
    'NIVEL_REGIONALIZAÇÃO'] = 'Região'

df_orcamento_r

In [ ]:
df_orcamento_r.loc[
    ~df_orcamento_r['SUBPREFEITURA'].str.contains('Supra', na=False),
    'NIVEL_REGIONALIZAÇÃO'] = 'Subprefeitura'

df_orcamento_r

In [ ]:
df_orcamento_r.loc[df_orcamento_r['NIVEL_REGIONALIZAÇÃO'].isna(), 'NIVEL_REGIONALIZAÇÃO'] = 'Não regionalizável'
df_orcamento_r

### Orçamento não regionalizado

In [ ]:
df_orcamento.head(1)

In [ ]:
cols_orcamento = ['Cd_SubFuncao', 'Ds_SubFuncao', 'Cd_Fonte']

cols_orcamento_vl = ['Vl_Orcado_Ano', 'Vl_Orcado_Atualizado', 'Vl_Liquidado']

df_orcamento_original = df_orcamento.copy()
df_orcamento = df_orcamento[cols_orcamento + cols_orcamento_vl]
df_orcamento

In [ ]:
df_orcamento = df_orcamento.groupby(cols_orcamento).sum().reset_index()
df_orcamento

In [ ]:
r_agg_cols = ['CÓDIGO_SUBFUNÇÃO', 'CÓDIGO_FONTE']

agg_cols = ['Cd_SubFuncao', 'Cd_Fonte']

df_orcamento_r_agg = (
    df_orcamento_r[r_agg_cols + ['VALOR_DETALHAMENTO_AÇÃO']]
    .groupby(r_agg_cols)
    .sum()
    .reset_index()
)

df_orcamento_r_agg.loc[:, 'VALOR_DETALHAMENTO_AÇÃO'] = (
    df_orcamento_r_agg
    .loc[:, 'VALOR_DETALHAMENTO_AÇÃO']
    .round(2)
)

df_orcamento_ajustado = df_orcamento.merge(
    df_orcamento_r_agg,
    left_on=agg_cols,
    right_on=r_agg_cols,
    how='left'
).drop(columns=r_agg_cols)

df_orcamento_ajustado

In [ ]:
df_orcamento_ajustado.loc[df_orcamento_ajustado['VALOR_DETALHAMENTO_AÇÃO'].isna(), 'VALOR_DETALHAMENTO_AÇÃO'] = 0

df_orcamento_ajustado.loc[:, 'Vl_Liquidado_N_Detalhado'] = (
    df_orcamento_ajustado.loc[:, 'Vl_Liquidado']
    - df_orcamento_ajustado.loc[:, 'VALOR_DETALHAMENTO_AÇÃO']).round(2)

df_orcamento_ajustado

In [ ]:
df_orcamento_ajustado[df_orcamento_ajustado['Vl_Liquidado_N_Detalhado']<0]

In [ ]:
df_orcamento_ajustado[['Vl_Liquidado', 'VALOR_DETALHAMENTO_AÇÃO']].sum()

### Unindo os dados de orçamento

Agora, vamos adicionar os dados não detalhados ao dataframe que contém o orçamento detalhado.

In [ ]:
orcamento_cols_map = {'Cd_SubFuncao': 'CÓDIGO_SUBFUNÇÃO',
                      'Ds_SubFuncao': 'DESCRIÇÃO_SUBFUNÇÃO',
                      'Cd_Fonte': 'CÓDIGO_FONTE',
                    #   'Ds_Fonte': 'DESCRIÇÃO_FONTE',
                      'Vl_Liquidado_N_Detalhado': 'Vl_Liquidado'}

df_orcamento_ajustado = (
    df_orcamento_ajustado
    .drop(columns=['VALOR_DETALHAMENTO_AÇÃO', 'Vl_Liquidado'])
    .rename(columns=orcamento_cols_map)
    )

df_orcamento_ajustado

In [ ]:
df_orcamento_r = (df_orcamento_r
                  .rename(columns={'VALOR_DETALHAMENTO_AÇÃO': 'Vl_Liquidado'}))

df_orcamento_r

In [ ]:
df_orcamento_final = pd.concat([df_orcamento_r, df_orcamento_ajustado])

df_orcamento_final

In [ ]:
df_orcamento_final.loc[df_orcamento_final['Vl_Orcado_Ano'].isna(),
                       'Vl_Orcado_Ano'] = 0
df_orcamento_final.loc[df_orcamento_final['Vl_Orcado_Atualizado'].isna(),
                       'Vl_Orcado_Atualizado'] = 0
df_orcamento_final.loc[df_orcamento_final['NIVEL_REGIONALIZAÇÃO'].isna(),
                       'NIVEL_REGIONALIZAÇÃO'] = 'Não detalhado'

df_orcamento_final

### Padronizando os nomes de subprefeituras

In [ ]:
subs_orcamento = (
    df_orcamento_final.loc[~df_orcamento_final['SUBPREFEITURA'].isna(), 'SUBPREFEITURA']
    .apply(unidecode)
    .unique()
    .tolist()
)

subs_orcamento.sort()

subs_orcamento

In [ ]:
subs_orcamento[:-6]

In [ ]:
len(subs_orcamento[:-6])

In [ ]:
subs_qlik = df_subs_qlik['sub.NOME'].unique().tolist()
subs_qlik.sort()
subs_qlik

In [ ]:
subs_qlik_orcamento = subs_qlik.copy()
# Guainases e Vila Prudente aparecem em uma ordenação diferente, por isso serão
# removidas e adicionadas novamente ao final da lista
subs_qlik_orcamento.remove('GUAIANASES')
subs_qlik_orcamento.remove('VILA PRUDENTE')
subs_qlik_orcamento.append('GUAIANASES')
subs_qlik_orcamento.append('VILA PRUDENTE')

In [ ]:
mapper_orcamento = {
    so: sq
    for so, sq in zip(subs_orcamento, subs_qlik_orcamento)
}

mapper_orcamento

In [ ]:
df_orcamento_final.insert(
    7,
    'sub.NOME',
    df_orcamento_final.loc[:,'SUBPREFEITURA'].apply(lambda s: unidecode(s) if isinstance(s, str) else None).map(mapper_orcamento)
)

df_orcamento_final

### Adicionando as subprefeituras faltantes

In [ ]:
orcamento_subs_cols = ['CÓDIGO_SUBFUNÇÃO', 'DESCRIÇÃO_SUBFUNÇÃO',
                       'CÓDIGO_FONTE']

df_orcamento_subs = df_orcamento_final[orcamento_subs_cols].copy()
df_orcamento_subs = df_orcamento_subs.drop_duplicates().reset_index(drop=True)
df_orcamento_subs

In [ ]:
df_orcamento_subs['TIPO_REGIONALIZAÇÃO'] = 'Despesa Regionalizável'
df_orcamento_subs['NIVEL_REGIONALIZAÇÃO'] = 'Subprefeitura'

df_orcamento_subs

In [ ]:
df_orcamento_subs = (
    df_orcamento_subs
    .merge(pd.DataFrame(columns=['sub.NOME'], data=subs_qlik),
           how='cross')
)

df_orcamento_subs

In [ ]:
df_orcamento_completo = (
    df_orcamento_final
    .merge(df_orcamento_subs,
           how='outer',
           on=df_orcamento_subs.columns.tolist())
)

df_orcamento_completo

In [ ]:
df_orcamento_completo.loc[df_orcamento_completo['Vl_Liquidado'].isna(), 'Vl_Liquidado'] = 0
df_orcamento_completo.loc[df_orcamento_completo['Vl_Orcado_Ano'].isna(), 'Vl_Orcado_Ano'] = 0
df_orcamento_completo.loc[df_orcamento_completo['Vl_Orcado_Atualizado'].isna(), 'Vl_Orcado_Atualizado'] = 0

df_orcamento_completo

### Adicionando descrição das vinculações

In [ ]:
df_orcamento_completo = (
    df_orcamento_completo
    .merge(df_orcamento_original[['Cd_Fonte', 'Ds_Fonte']].drop_duplicates(),
            how='left',
            left_on='CÓDIGO_FONTE',
            right_on='Cd_Fonte')
    .drop(columns='Cd_Fonte')
)

df_orcamento_completo

# Armazenamento dos dados

Finalmente, salvamos os arquivos como csv para utilizarmos no Qlik Sense.

In [ ]:
base_path = os.path.join('data_output', 'educacao')

if not os.path.exists(base_path):
    os.makedirs(base_path)

for name, df in [('demanda e matriculas', mat_24),
                 ('servidores-perfil', perf_24),
                 ('escolas-municipais', escolas_24),
                 ('perfil-dos-educandos', alunos_24),
                 ('educandos-estrangeiros', alunos_est_2024),
                 ('beneficiarios-pbf', alunos_bf_2024),
                 ('nivel-padronizado', nivel_padronizado),
                 ('orcamento', df_orcamento_completo)]:
    
    filepath = os.path.join(base_path, f'{name}.csv')

    df.to_csv(filepath,
              index=False,
              sep=';',
              decimal=',',
              encoding='utf-8')